In [ ]:
#Requiered libraries
import pandas as pd
import os
import numpy as np
from datetime import date
import datetime
from keras.utils import image_dataset_from_directory
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

from tensorflow.keras.layers import Rescaling
from tensorflow.keras.layers import Resizing
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.layers import RandomZoom
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.layers import RandomBrightness
from tensorflow.keras.layers import RandomContrast 
from tensorflow.keras.layers import RandomTranslation

# For evaluating performance
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

#Tensorboard
import io
import itertools
from packaging import version
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model
import tensorflow as tf
import datetime
%load_ext tensorboard

In [ ]:
#Reading the dataset


DATE= date.today().strftime("%Y_%m_%d")
DATASET_PATH_train = r"C:\Users\Alex\Documents\GitHub\may25_bds_plants\05_data\original_data\2.1.1 New Plant Diseases\train"

DATASET_PATH_valid = r"C:\Users\Alex\Documents\GitHub\may25_bds_plants\05_data\original_data\2.1.1 New Plant Diseases\valid"

#Constant variables

IMAGE_SIZE = (128,128, 3)  # (height, width, channels)
vector_size_1D = IMAGE_SIZE[0] * IMAGE_SIZE[1] * IMAGE_SIZE[2]  # Flattened size for each image
batch_size = 64

train = image_dataset_from_directory(
    DATASET_PATH_train,  # Path to the dataset
    labels='inferred',  # Automatically infer labels from subdirectory names
    #label_mode='categorical',  # Use categorical labels
    image_size=IMAGE_SIZE[:2],  # Resize images to the specified size
    batch_size=batch_size,  # Number of images per batch
    seed=42,  # Random seed for reproducibility
    )

class_names = train.class_names #Saves all class names in a list 
classes_healthy = [class_name for class_name in train.class_names if "healthy" in class_name.lower()]
classes_sick = [class_name for class_name in train.class_names if "healthy" not in class_name.lower()]

# Validation dataset
valid = image_dataset_from_directory(
    DATASET_PATH_valid,  # Path to the dataset
    labels='inferred',  # Automatically infer labels from subdirectory names
    #label_mode='categorical',  # Use categorical labels
    image_size=IMAGE_SIZE[:2],  # Resize images to the specified size
    batch_size=batch_size,  # Number of images per batch
    seed=42,  # Random seed for reproducibility
    )

dataset = train #Choose the dataset to work with, e.g., train, valid, test_healthy, test_sick
dataset_valid = valid #Choose the dataset to work with, e.g., train, valid, test_healthy, test_sick
dataset.class_names = sorted(dataset.class_names)  # Sort class names for consistency

# Configuration for performance optimization
# This is used to optimize the performance of data loading and preprocessing
AUTOTUNE = tf.data.AUTOTUNE
# Apply performance optimizations to the datasets
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
dataset_valid = dataset_valid.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:

#load the model

#model_path = r"C:\Users\Alex\Documents\GitHub\may25_bds_plants\05_data\models\model_2025_07_29.keras"
model_path = 
model = load_model(model_path)

In [ ]:
# Create evaluation data
#Just run

def get_predictions_and_labels(dataset):
    true_labels = []
    pred_labels = []

    for images, labels in dataset:
        
        preds = model.predict(images, verbose=0)  # Get the model's predictions
        pred_labels.extend(np.argmax(preds, axis=-1))  # Get the predicted labels (argmax)

        true_labels.extend(labels.numpy())  # Get the true labels

    return np.array(true_labels), np.array(pred_labels)

def generate_classification_report(y_true, y_pred):
    return classification_report(y_true, y_pred, output_dict=True)

def plot_confusion_matrix(y_true, y_pred, class_names=None):
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", cbar=False,
                xticklabels=class_names, yticklabels=class_names, ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title("Normalized Confusion Matrix")
    return fig


y_true, y_pred = get_predictions_and_labels(dataset_valid)  # Get true labels and predictions from the test dataset

# Display the classification report
print(classification_report(y_true, y_pred))  # Print the classification report (precision, recall, F1-score)

# Display the confusion matrix
plt.figure(figsize=(15, 15))  # Create a large figure for the confusion matrix
cnf_matrix = confusion_matrix(y_true, y_pred, normalize='true')  # Compute the normalized confusion matrix
sns.heatmap(cnf_matrix, cmap='Blues', annot=True, cbar=False, fmt=".2f")  # Plot the confusion matrix as a heatmap

In [ ]:
report = classification_report(y_true, y_pred, output_dict=True)
cmatrix = confusion_matrix(y_true, y_pred, normalize='true')

#report_file = "first_model_classification_results_2025_07_29.pkl"
report_file = 
with open(report_file, "wb") as f:
    pickle.dump({
        "y_true": y_true,
        "y_pred": y_pred,
        "report": report,
        "confusion_matrix": cmatrix
    }, f)


In [ ]:
# Grad-cam file
#just run
#Grad-CAM: Visualizing Decisions

#Selection 4 images from the validation dataset
X = np.array([img.numpy() for img, _ in dataset_valid.take(2)][0]).astype(np.uint8)
y = np.array([label.numpy() for _, label in dataset_valid.take(2)][0])

# Select specific images
images = X[:4]
labels = y[:4]

def grad_cam(image, model, layer_name):
    if not any(layer.name == layer_name for layer in model.layers):
        raise ValueError(f"Layer {layer_name} not found in model.")
    # Retrieve the convolutional layer
    layer = model.get_layer(layer_name)
    
    # Create a model that generates the outputs of the convolutional layer and the predictions
    grad_model = Model(inputs=model.input, outputs=[layer.output, model.output])

    # Add a batch dimension
    image = tf.expand_dims(image, axis=0)

    # Compute the gradients
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(image)
        predicted_class = tf.argmax(predictions[0])  # Predicted class
        loss = predictions[:, predicted_class]  # Loss for the predicted class

    # Gradients of the scores with respect to the outputs of the convolutional layer
    grads = tape.gradient(loss, conv_outputs)

    # Weighted average of the gradients for each channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Weight the activations by the calculated gradients
    conv_outputs = conv_outputs[0]  # Remove the batch dimension
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    # Normalize the heatmap
    heatmap = tf.maximum(heatmap, 0)  # Focus only on positive values
    heatmap /= tf.math.reduce_max(heatmap) + 1e-6  # Normalize between 0 and 1
    heatmap = heatmap.numpy()  # Convert to numpy array for visualization

    # Resize the heatmap to match the original image size
    heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], (image.shape[1], image.shape[2])).numpy()
    heatmap_resized = np.squeeze(heatmap_resized, axis=-1)  # Remove the singleton dimension at the end of the heatmap_resized array

    # Color the heatmap with a palette (e.g., "jet")
    heatmap_colored = plt.cm.jet(heatmap_resized)[..., :3]  # Get the R, G, B channels 

    superimposed_image = heatmap_colored * 0.7 + image[0].numpy() / 255.0

    return np.clip(superimposed_image, 0, 1), predicted_class

import os
from PIL import Image

def format_class_name(label):
    plant, disease = label.split("___")
    return f"{plant} ({disease.replace('_', ' ')})"

def show_grad_cam_cnn(images, labels, model, class_names, save_dir=None):
    os.makedirs(save_dir, exist_ok=True) if save_dir else None

    number_of_images = images.shape[0]
    conv_layers = [layer.name for layer in model.layers if isinstance(layer, Conv2D)]
    conv_layers = conv_layers[-1:]  # Use only the last convolutional layer for Grad-CAM
    
    plt.figure(figsize=(16,16))

    for j, layer in enumerate(conv_layers):

        for i in range(number_of_images):

            subplot_index = i + 1 + j * number_of_images
            plt.subplot(len(conv_layers), number_of_images, subplot_index)

            # Get the image with the overlaid heatmap
            grad_cam_image, predicted_class = grad_cam(images[i], model, layer)
            true_label = format_class_name(class_names[labels[i]])
            predicted_label = format_class_name(class_names[predicted_class])
            
            # Display the image with Grad-CAM
            #plt.title(f'Grad-CAM {layer}')
            plt.title(f'True: {true_label}\nPred: {predicted_label}', fontsize=10)
            plt.imshow(grad_cam_image)
            plt.axis("off")

            if save_dir:
                filename = f"{class_names[labels[i]]}_img{i+1}.jpg"
                filepath = os.path.join(save_dir, filename)
                gradcam_uint8 = (grad_cam_image * 255).astype(np.uint8)
                Image.fromarray(gradcam_uint8).save(filepath)

    if not save_dir:
        plt.show()
#-------------------------------------------------------------
#Damit wir zwei Bilder pro Klasse haben, iterieren wir durch das Validierungsset
from collections import defaultdict

# Ziel: Zwei Bilder pro Klasse (z. B. 'Apple___healthy', 'Tomato___Early_blight', …)
images_by_class = defaultdict(list)

# Iteriere durch das gesamte Validierungsset
for images, labels in dataset_valid:
    for img, label in zip(images, labels):
        label_int = int(label.numpy())
        if len(images_by_class[label_int]) < 2:
            images_by_class[label_int].append(img.numpy())
        # Abbruchbedingung: wenn alle Klassen 2 Bilder haben
        if all(len(v) >= 2 for v in images_by_class.values()):
            break
    if all(len(v) >= 2 for v in images_by_class.values()):
        break

# Jetzt alle gesammelten Bilder & Labels extrahieren
all_images = []
all_labels = []

for label, imgs in images_by_class.items():
    all_images.extend(imgs)
    all_labels.extend([label] * len(imgs))

# In Arrays umwandeln
all_images = np.array(all_images).astype(np.uint8)
all_labels = np.array(all_labels)


In [ ]:
#show_grad_cam_cnn(images, model)
#save_path = r"C:\Users\Alex\Documents\GitHub\may25_bds_plants\04_src\images_grad_cam\first_model_2025_07_29"
save_path = 
#show_grad_cam_cnn(images, labels, model, class_names, save_dir=save_path) #für 4 bilder 

show_grad_cam_cnn(all_images, all_labels, model, class_names, save_dir=save_path) #für alle bilder


In [ ]:
import shap
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from collections import defaultdict

def explain_and_save_overlay_per_class(
    model,
    dataset,                 # tf.data.Dataset
    class_names,
    save_dir,
    max_evals=500,
    samples_per_class=2
):
    os.makedirs(save_dir, exist_ok=True)

    # Bilder pro Klasse sammeln
    collected = defaultdict(list)

    for image, label in dataset.unbatch():
        label_index = int(label.numpy())
        if len(collected[label_index]) < samples_per_class:
            collected[label_index].append(image.numpy())
        # Stop early if all classes are full
        if all(len(v) >= samples_per_class for v in collected.values()):
            break

    # Pro Klasse SHAP generieren
    for class_index, images in collected.items():
        class_name = class_names[class_index]
        images = np.stack(images, axis=0)

        print(f"\n🔍 Klasse: {class_name} | Anzahl Bilder: {len(images)}")

        # SHAP-Erklärer
        masker = shap.maskers.Image("inpaint_telea", images[0].shape)
        explainer = shap.Explainer(model, masker, output_names=class_names)

        shap_values = explainer(
            images,
            max_evals=max_evals,
            outputs=shap.Explanation.argsort.flip[:1]
        )

        for i, shap_exp in enumerate(shap_values):
            img = images[i]

            shap_val = shap_exp.values
            if isinstance(shap_val, list):
                shap_val = shap_val[0]

            while shap_val.ndim > 3:
                shap_val = np.squeeze(shap_val, axis=-1)

            if shap_val.shape != img.shape:
                print(f"❌ Shape mismatch: {shap_val.shape} vs {img.shape}")
                continue

            # SHAP-Karte mitteln (H, W)
            shap_overlay = shap_val.mean(axis=-1)

            # Normalisieren
            shap_norm = (shap_overlay - shap_overlay.min()) / (shap_overlay.ptp() + 1e-8)
            cmap = plt.get_cmap("bwr")
            heatmap = cmap(shap_norm)[:, :, :3]

            img_float = img.astype(np.float32) / 255.0
            overlay = (0.6 * img_float + 0.4 * heatmap).clip(0, 1)

            # Speichern
            safe_class_name = class_name.replace('/', '_').replace(' ', '_')
            overlay_path = os.path.join(save_dir, f"{safe_class_name}_img{i+1}_overlay.png")
            original_path = os.path.join(save_dir, f"{safe_class_name}_img{i+1}_original.png")

            Image.fromarray((overlay * 255).astype(np.uint8)).save(overlay_path)
            Image.fromarray(img.astype(np.uint8)).save(original_path)

            print(f"✅ Gespeichert: {overlay_path}")
            print(f"✅ Gespeichert: {original_path}")


In [ ]:
#save_path_shap = r"C:\Users\Alex\Documents\GitHub\may25_bds_plants\04_src\images_shap\first_model_2025_07_29"
save_path_shap = 


explain_and_save_overlay_per_class(
    model=model,
    dataset=dataset_valid,
    class_names=class_names,
    save_dir=save_path_shap,
    max_evals=500,
    samples_per_class=2  # 2 Bilder pro Klasse
)